# Algorytm Apriori

In [1]:
from itertools import combinations
import pandas as pd

In [ ]:
# 1. Implementacja ręczna algorytmu Apriori:

# Dane wejściowe
data = pd.DataFrame({
    'A': [1, 1, 0, 1, 0],
    'B': [1, 0, 1, 1, 1],
    'C': [0, 1, 1, 0, 1],
    'D': [1, 0, 1, 0, 1]
})

print(data)

# Parametry algorytmu
min_support = 0.4
min_confidence = 0.6

# Obliczanie wsparcia dla zbiorów elementów
def calculate_support(data, itemset):
    count = 0
    for i in range(len(data)):
        if all(data.iloc[i][item] == 1 for item in itemset):
            count += 1
    return count / len(data)

# Generowanie  zbiorów częstych
def apriori(data, min_support):
    columns = list(data.columns)
    frequent_itemsets = []  # Lista na zbiory częste

    # Zbiory jednoelementowe
    for col in columns:
        support = calculate_support(data, [col])
        if support >= min_support:
            frequent_itemsets.append(([col], support))

    # Iteracyjne generowanie większych zbiorów
    k = 2
    while True:
        new_combinations = list(combinations([item[0] for item in frequent_itemsets if len(item[0]) == k - 1], k))
        new_frequent = []
        for combo in new_combinations:
            merged = set().union(*combo)
            if len(merged) == k:
                support = calculate_support(data, merged)
                if support >= min_support:
                    new_frequent.append((list(merged), support))
        if not new_frequent:
            break
        frequent_itemsets.extend(new_frequent)
        k += 1

    return frequent_itemsets

# Generowanie reguł asocjacyjnych
def generate_rules(frequent_itemsets, min_confidence):
    rules = []
    for itemset, support in frequent_itemsets:
        if len(itemset) > 1:
            for i in range(1, len(itemset)):
                for antecedent in combinations(itemset, i):
                    consequent = set(itemset) - set(antecedent)
                    if consequent:
                        antecedent = list(antecedent)
                        consequent = list(consequent)
                        antecedent_support = calculate_support(data, antecedent)
                        confidence = support / antecedent_support
                        if confidence >= min_confidence:
                            rules.append({
                                'antecedent': antecedent,
                                'consequent': consequent,
                                'support': support,
                                'confidence': confidence
                            })
    return rules

# Wykonanie algorytmu
frequent_itemsets = apriori(data, min_support)
rules = generate_rules(frequent_itemsets, min_confidence)

# Wyświetlanie wyników
print("Zbiory częste:")
for itemset, support in frequent_itemsets:
    print(f"{itemset}: Wsparcie = {support:.2f}")

print("\nReguły asocjacyjne:")
for rule in rules:
    antecedent = ', '.join(rule['antecedent'])
    consequent = ', '.join(rule['consequent'])
    print(f"{antecedent} => {consequent} (Wsparcie = {rule['support']:.2f}, Ufność = {rule['confidence']:.2f})")

   A  B  C  D
0  1  1  0  1
1  1  0  1  0
2  0  1  1  1
3  1  1  0  0
4  0  1  1  1
Zbiory częste:
['A']: Wsparcie = 0.60
['B']: Wsparcie = 0.80
['C']: Wsparcie = 0.60
['D']: Wsparcie = 0.60
['A', 'B']: Wsparcie = 0.40
['C', 'B']: Wsparcie = 0.40
['D', 'B']: Wsparcie = 0.60
['D', 'C']: Wsparcie = 0.40
['D', 'C', 'B']: Wsparcie = 0.40

Reguły asocjacyjne:
A => B (Wsparcie = 0.40, Ufność = 0.67)
C => B (Wsparcie = 0.40, Ufność = 0.67)
D => B (Wsparcie = 0.60, Ufność = 1.00)
B => D (Wsparcie = 0.60, Ufność = 0.75)
D => C (Wsparcie = 0.40, Ufność = 0.67)
C => D (Wsparcie = 0.40, Ufność = 0.67)
D => C, B (Wsparcie = 0.40, Ufność = 0.67)
C => D, B (Wsparcie = 0.40, Ufność = 0.67)
D, C => B (Wsparcie = 0.40, Ufność = 1.00)
D, B => C (Wsparcie = 0.40, Ufność = 0.67)
C, B => D (Wsparcie = 0.40, Ufność = 1.00)
